In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import feature
from skimage.morphology import binary_erosion
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture

In [5]:
def showImage(img):
    plt.imshow(img, cmap='gray')
    plt.show()

def getBorderColor(img):
    
    border = np.asarray(img[0,:])
    border = np.concatenate((border, np.asarray(img[-1, :])))
    border = np.concatenate((border, np.asarray(img[:, 0])))
    border = np.concatenate((border, np.asarray(img[:, -1])))
    return np.bincount(border).argmax()

def pre_processing(img):
    _, img_binarized = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)    
    if(getBorderColor(img_binarized) != 0):
        img_binarized = cv2.bitwise_not(img_binarized) 
    return img_binarized

In [6]:
def getFeaturesHOG(img):
   img = cv2.resize(img, (256, 128))
   cell_size = (32, 32)  # h x w in pixels
   block_size = (2, 2)  # h x w in cells
   nbins = 9  # number of orientation bins
   # winSize is the size of the image cropped to an multiple of the cell size
   hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                    img.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)

   hog_feats = hog.compute(img)
   return hog_feats.flatten() 

In [8]:
def get_center_of_mass(window):
    # calculate moments of binary image
    M = cv2.moments(window)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / (M["m00"] + 1e-5))
    cY = int(M["m01"] / (M["m00"] + 1e-5))
    return cX,cY

# pre processing of the image (baseline estimation)
def baseline_estimatator(img):
    horz_proj = np.sum(img, 1)
    lb = np.argmax(horz_proj)
    avg_row_density = np.mean(horz_proj)
    lu = np.argmax(horz_proj >= avg_row_density)
    return lb, lu


def leftUp(img, height):
    k1 = np.array([
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 0]
    ])
    k2 = np.array([
        [0, 0, 1],
        [0, 1, 1],
        [1, 1, 0]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def rightUp(img, height):
    k1 = np.array([
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 0]
    ])
    k2 = np.array([
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 1]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def rightDown(img, height):
    k1 = np.array([
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0]
    ])
    k2 = np.array([
        [0, 1, 1],
        [1, 1, 0],
        [1, 0, 0]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def leftDown(img, height):
    k1 = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0]
    ])
    k2 = np.array([
        [1, 1, 0],
        [0, 1, 1],
        [0, 0, 1]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def vertical(img, height):
    k1 = np.array([
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]
    ])
    k2 = np.array([
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def horizontal(img, height):
    k1 = np.array([
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 1]
    ])
    k2 = np.array([
        [0, 0, 0],
        [1, 1, 1],
        [0, 0, 0]
    ])
    return np.sum(binary_erosion(255 - img, k2) * binary_erosion(img, k1))


def get_center_of_mass(window):
    # calculate moments of binary image
    M = cv2.moments(window)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / (M["m00"] + 1e-5))
    cY = int(M["m01"] / (M["m00"] + 1e-5))
    return cX, cY

def getImageFeatures(img, WIDTH, HEIGHT, STRID, N, baseline):
    lb, lu = baseline
    img_height = img.shape[0]
    img_width = img.shape[1]
    X = []
    centers = []
    for r in range(img_width - 1, WIDTH - 1, -STRID):
        window = img[:, r-WIDTH:r]
        X_WINDOW = []
        centers.append(get_center_of_mass(window))
        X_WINDOW.append(np.sum(img[0:lb-1, r-WIDTH:r] == 255) / (img_height * WIDTH))
        X_WINDOW.append(np.sum(img[lb:, r-WIDTH:r] == 255) /
                  ((img_height - lb) * WIDTH))
        X_WINDOW.append(np.sum(window == 255) / (WIDTH * img_height))
        X_WINDOW.append(leftUp(window, img_height))
        X_WINDOW.append(rightUp(window, img_height))
        X_WINDOW.append(rightDown(window, img_height))
        X_WINDOW.append(leftDown(window, img_height))
        X_WINDOW.append(vertical(window, img_height))
        X_WINDOW.append(horizontal(window, img_height))
        window = img[lu:lb, r-WIDTH:r]
        X_WINDOW.append(leftUp(window, np.abs(lb-lu)))
        X_WINDOW.append(rightUp(window, np.abs(lb-lu)))
        X_WINDOW.append(rightDown(window, np.abs(lb-lu)))
        X_WINDOW.append(leftDown(window, np.abs(lb-lu)))
        X_WINDOW.append(vertical(window, np.abs(lb-lu)))
        X_WINDOW.append(horizontal(window, np.abs(lb-lu)))
        window = img[lu:lb, r-WIDTH:r]
        for w in range(WIDTH):
            X_WINDOW.append(np.sum(window[:, w]) / img_height)
        contours, _ = cv2.findContours(
            window.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        X_WINDOW.append(len(contours))
        X_WINDOW.append(np.average(cv2.Canny(window, 50, 200) == 255))
        X_WINDOW.append(len(contours))
        X.append(X_WINDOW)
    X[0].append(centers[0][0])
    X[0].append(centers[0][1])
    for i in range(1, len(centers)):
        X[i].append(centers[i][0] - centers[i - 1][0])
        X[i].append(centers[i][1] - centers[i - 1][1])
    for i in range(0, len(centers)):
        X[i].append((centers[i][1] - lb) / img_height)
    for inx, vec in enumerate(X):
        X[inx] = np.array(vec)
    
    return np.array(X, dtype=object)


In [17]:
data_set = []
Y = []
for i in range(1, 10):
    for filename in os.listdir("ACdata_base/" + str(i)):
        img = cv2.imread(os.path.join("ACdata_base/" + str(i),
                                      filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            data_set.append(pre_processing(img))
            Y.append(i)
X_train, X_test, Y_train, Y_test = train_test_split(data_set, Y, test_size=0.2, random_state=42)

# X_validation, X_test, Y_validation, Y_test = train_test_split( X_testValid, Y_testValid, test_size=0.5, random_state=42)
# showImage(data_set[2])
def getFeatures28(images):
    N = 20
    WIDTH = 32 
    STRID = 8
    x_features = []
    for i in range(len(images)):
            img = images[i]
            x_features.append((getImageFeatures(img, WIDTH, int(img.shape[0] / N), STRID, N, baseline_estimatator(img)), Y_train[i]))
    return np.asarray(x_features, dtype=object)


features28_training = getFeatures28(X_train)
# features28_test = getFeatures28(X_test)

print(features28_training.shape)


# print(features28_test.shape)


(1348, 2)


In [18]:
X = [[] for _ in range(9)]
for (mat, inx) in features28_training:
    for vec in mat:
        X[inx - 1].append(vec)


In [19]:
X1 = np.array(X[0])
print(X1.shape)
X2 = np.array(X[1])
print(X2.shape)
X3 = np.array(X[2])
print(X3.shape)
X4 = np.array(X[3])
print(X4.shape)
X5 = np.array(X[4])
print(X5.shape)
X6 = np.array(X[5])
print(X6.shape)
X7 = np.array(X[6])
print(X7.shape)
X8 = np.array(X[7])
print(X8.shape)
X9 = np.array(X[8])
print(X9.shape)

(7278, 53)
(3409, 53)
(4425, 53)
(5166, 53)
(8073, 53)
(6194, 53)
(5494, 53)
(4258, 53)
(6101, 53)


In [20]:
class Model:
    CLASSES = 9
    def __init__(self):
        self.models = [GaussianMixture(n_components=6, max_iter=200)
                       for _ in range(Model.CLASSES)]
    def fit(self, X, y):
        self.models[y].fit(X)
    def pridect(self, X):
        label = None
        global_likelihood = float('-inf')
        for inx, model in enumerate(self.models):
            local_likelihoods = model.score_samples(X)
            temp_global_likelihood = sum(local_likelihoods)

            if temp_global_likelihood > global_likelihood:
                label = inx
                global_likelihood = temp_global_likelihood
        return label

In [21]:
M = Model()
M.fit(X1, 0)
M.fit(X2, 1)
M.fit(X3, 2)
M.fit(X4, 3)
M.fit(X5, 4)
M.fit(X6, 5)
M.fit(X7, 6)
M.fit(X8, 7)
M.fit(X9, 8)

In [22]:
img = X_test[3]
X_t = getImageFeatures(img, 32, int(img.shape[0] / 20), 16, 20, baseline_estimatator(img))
    
print(Y_test[3])
print(M.pridect(X_t) + 1)


1
1


In [23]:
ac = 0
for inx, img in enumerate(X_train):
    X_t = getImageFeatures(img, 32, int(
        img.shape[0] / 20), 16, 20, baseline_estimatator(img))
    if M.pridect(X_t) + 1 == Y_train[inx]:
        ac +=1
print(ac/len(Y_train) * 100)


97.99703264094956


In [24]:
ac = 0
for inx, img in enumerate(X_test):
    X_t = getImageFeatures(img, 32, int(
        img.shape[0] / 20), 16, 20, baseline_estimatator(img))
    if M.pridect(X_t) + 1 == Y_test[inx]:
        ac += 1
print(ac/len(Y_test) * 100)


92.87833827893175
